In [ ]:
"""Notebook to calculate the yearly compounding returns of
the strategy after accounting for 25% short term capital
gains tax, but not accounting for any trading fees"""

In [1]:
import sqlalchemy
import pandas as pd
engine = sqlalchemy.create_engine('sqlite:///backtesting.db')
with engine.connect() as con:
    sp500_monthly_returns = pd.read_sql(
        'sp500_monthly_returns', con=con, index_col='Date')
    strategy_monthly_returns = pd.read_sql(
        'strategy_monthly_returns', con=con, index_col='Date')

In [2]:
# Replacing any monthly return over 5x with np.nan
def remove_outliers(df_):
    return (df_.where(lambda x: x<=5))

sp_clean = remove_outliers(sp500_monthly_returns)
strat_clean = remove_outliers(strategy_monthly_returns)

In [3]:
# Creating a dataframe with the mean monthly returns of
# the sp500 and of the strategy selection, and calling it 
# df_joined
sp_means = sp_clean.mean(axis=1)
strat_means = strat_clean.mean(axis=1)
df_joined = pd.concat([strat_means, sp_means],
 axis=1)
df_joined.columns= 'strat', 'sp500'

In [4]:
# Calculating the yearly returns after removing short term tax each year
yearly_returns_taxed = (df_joined
    # loc on strat values where the previous sp500 return was >=1
    .loc[(df_joined["sp500"].shift(-1)>=1), ["strat"]]
    # Calculate yearly returns
    .resample('Y').prod()
    .squeeze()
    # For positive returns, scale growth of return by 75% to account 
    # for short term capital gains tax
    .apply(lambda x: (((x-1)*0.75)+1) if x>1 else x))

In [5]:
# Calculate the yearly coompunding returns of strategy
yearly_returns_taxed.cumprod()

Date
2000-12-31    1.477622
2001-12-31    1.565753
2002-12-31    1.361841
2003-12-31    1.962405
2004-12-31    2.173993
2005-12-31    2.279926
2006-12-31    1.772903
2007-12-31    2.169815
2008-12-31    1.690541
2009-12-31    1.905637
2010-12-31    2.217822
2011-12-31    2.177991
2012-12-31    2.438576
2013-12-31    2.813285
2014-12-31    3.011508
2015-12-31    2.868464
2016-12-31    3.377878
2017-12-31    3.969569
2018-12-31    3.300014
2019-12-31    3.544506
2020-12-31    4.737846
2021-12-31    4.668606
2022-12-31    4.104288
Freq: A-DEC, Name: strat, dtype: float64